In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
 ! pip install -q kaggle

In [ ]:
'''import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/kaggle''''

In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"marmoh","key":"7ef6c3cd0b867678fcb9271d320218dd"}'}

In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
 ! kaggle datasets list

ref                                                             title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
rohit265/credit-card-eligibility-data-determining-factors       Credit Card Eligibility Data: Determining Factors  296KB  2024-05-18 11:31:28           1103         24  1.0              
rahulvyasm/netflix-movies-and-tv-shows                          Netflix Movies and TV Shows                          1MB  2024-04-10 09:48:38          25843        551  1.0              
mayankanand2701/netflix-stock-price-dataset                     Netflix Stock Price Dataset 🎥🍿🎬📊                   107KB  2024-05-25 06:59:35            708         32  1.0              
prasad22/vehicle-population-statistics                          V

In [ ]:
! kaggle datasets download -d muratkokludataset/rice-image-dataset

Dataset URL: https://www.kaggle.com/datasets/muratkokludataset/rice-image-dataset
License(s): CC0-1.0
 98% 214M/219M [00:02<00:00, 119MB/s]
100% 219M/219M [00:02<00:00, 101MB/s]


In [ ]:
import zipfile

# Define the path to your zip file
file_path = '/content/rice-image-dataset.zip'

# Unzip the file to a specific destination
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/kaggle')

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.applications.mobilenet_v2 import decode_predictions

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [ ]:

pip install split-folders


In [ ]:
! mkdir ~/.test

In [ ]:
import splitfolders
input_folder = "/content/kaggle/Rice_Image_Dataset"
output = "/content/testt"

splitfolders.ratio(input_folder, output=output, seed=42, ratio=(.8, .2))

Copying files: 75000 files [00:14, 5150.01 files/s]


In [ ]:
base_dir_path = os.getcwd()
print(base_dir_path)
train_dir_path = os.path.join(base_dir_path,'testt/train')
print(train_dir_path)
test_dir_path = os.path.join(base_dir_path,'testt/val')
MNet_InputSize = (224,224)

/content
/content/testt/train


In [ ]:
def getAllClassNames(dir_path):
    return os.listdir(dir_path)

In [ ]:
def understandData(BASE_DIR_PATH,train_or_test):
    """
    Function prints number of images per class in train/test directory
    <CLASS-NAME    NUMBER-OF-IMAGES>

    Args:
        BASE_DIR_PATH(str): path of the base directory
        train_or_test(str): directory to select train/test
    """
    train_dir_path = os.path.join(BASE_DIR_PATH,train_or_test)
    test_dir_path = os.path.join(BASE_DIR_PATH,'testt/val')
    print("Number of Classes = ",len(os.listdir(train_dir_path)))
    AllClassNames = os.listdir(train_dir_path)
    print("Class Names = ",AllClassNames)
    print('CLASS NAME'+'\t'+'NUMBER OF IMAGES')
    for class_name in AllClassNames:
        print(class_name+'\t',len(os.listdir(os.path.join(train_dir_path,class_name))))
    displaySampleImages(train_dir_path,AllClassNames)
    return

In [ ]:
def displaySampleImages(PATH_TO_DIR,ALL_CLASS_NAMES):
    """
    Display grid of sample images for every class in dataset.

    Args:
        PATH_TO_DIR(str): path to train or test dir.
        ALL_CLASS_NAMES(str): list of all class names.

    """
    import matplotlib.pyplot as plt
    import matplotlib as mpl
    mpl.rcParams['axes.titlesize'] = 8
    import glob
    import cv2
    #NoOfClasses = len(ALL_CLASS_NAMES)
    fig = plt.figure()
    fig.subplots_adjust(hspace=0.7, wspace=0.1)
    fig.suptitle('Understanding Fruit-360 Dataset', fontsize=16)
    for n,class_name in enumerate(ALL_CLASS_NAMES):
        ImagePath = glob.glob(os.path.join(PATH_TO_DIR,class_name)+'/*.jpg')[0]
        #print(ImagePath)
        Img = cv2.imread(ImagePath)
        ax = fig.add_subplot(10,10,(n+1))
        plt.imshow(cv2.cvtColor(Img, cv2.COLOR_BGR2RGB))
        ax.set_title(class_name+str(n))
        plt.axis('off')
    plt.show()
    return

In [ ]:
def readData(BASE_DIR_PATH):
    """
    Console output of,
        total number of classes in train/test dir
        total number of images in train/test dir
    in given dataset.

    Args:
        BASE_DIR_PATH(str): path to root dir
    """
    nb_of_train_files = 0
    nb_of_test_files = 0

    train_dir_path = os.path.join(BASE_DIR_PATH,'testt/train')
    test_dir_path = os.path.join(BASE_DIR_PATH,'testt/val')
    AllClassNames_train = os.listdir(train_dir_path)
    AllClassNames_test = os.listdir(test_dir_path)
    print('Number of Classes in train DataSet: ',len(AllClassNames_train))
    print('Number of Classes in test DataSet: ',len(AllClassNames_test))
    for class_name in AllClassNames:
        nb_of_train_files = nb_of_train_files + len(os.listdir(os.path.join(train_dir_path,class_name)))
        nb_of_test_files = nb_of_test_files + len(os.listdir(os.path.join(test_dir_path,class_name)))
    print('Number of train samples: ',nb_of_train_files)
    print('Number of test samples:',nb_of_test_files)
    return

In [ ]:
def id_class_name(class_id, classes):
    """
    Returns name of the class as per the given id

    Args:
        class_id(int): Number of the class.
        classes(dict): dictinary of all the classes in given dataset.

    returns:
        Name of the Class.
    """
    for key, value in classes.items():
        if class_id == key:
            return value

In [ ]:
def predictRiceClass(ImagePath,trainedModel,DictOfClasses):
    """
    Perform class prediction on input image and print predicted class.

    Args:
        ImagePath(str): Absolute Path to test image
        trainedModel(object): trained model from method getTrainedModel()
        DictOfClasses(dict): python dict of all image classes.

    Returns:
        Probability of predictions for each class.

    """
    x = image.load_img(ImagePath, target_size=MNet_InputSize)
    x = image.img_to_array(x)
    #for Display Only
    import matplotlib.pyplot as plt
    plt.imshow((x * 255).astype(np.uint8))
    x = np.expand_dims(x,axis=0)
    x = preprocess_input(x)
    prediction_class = trainedModel.predict_classes(x,batch_size=1)
    prediction_probs = trainedModel.predict_proba(x,batch_size=1)
    #print(prediction_probs)
    class_value = id_class_name(prediction_class,DictOfClasses)
    print(class_value)
    return prediction_probs

In [ ]:
def getTrainedModel(PATH_TO_TRAINED_MODEL_FILE):
    """
    Loads trained-saved model from file(.h5) and returns as a object.

    Args:
        PATH_TO_TRAINED_MODEL_FILE(str): path to saved model file.

    returns:
        trainedModel(model object): returns a model saved as a <.h5>
    """
    from tensorflow.keras.models import load_model
    traiendModel = load_model(PATH_TO_TRAINED_MODEL_FILE)
    return traiendModel

In [ ]:
def plotResults(historyObject):
    """
    Summary:
        Plots train-validation loss and accuracy graphs for given history object

    Args:
        historyObject(object): Object returned by fit/fit_generator during train.
    """
    import matplotlib.pyplot as plt
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1)
    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.figure()
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    plt.show()
    return


In [ ]:
AllClassNames = getAllClassNames(train_dir_path)
num_of_classes = len(AllClassNames)
DictOfClasses = {i : AllClassNames[i] for i in range(0, len(AllClassNames))}

In [ ]:
conv_base = MobileNetV2(weights='imagenet',
                        include_top=False,
                        pooling='avg')
conv_base.trainable = False
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_of_classes,activation='softmax'))

optimizer = Adam(lr=1e-5)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 1280)              2257984   
 tional)                                                         
                                                                 
 flatten_1 (Flatten)         (None, 1280)              0         
                                                                 
 dense_2 (Dense)             (None, 512)               655872    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 5)                 2565      
                                                                 
Total params: 2916421 (11.13 MB)
Trainable params: 658437 (2.51 MB)
Non-trainable params: 2257984 (8.61 MB)
____________

In [ ]:
datagen = ImageDataGenerator(rescale=1./255,
                             shear_range=0.4,
                             zoom_range=0.2,
                             rotation_range=50,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             horizontal_flip=True,
                             fill_mode='nearest')





In [ ]:
batch_size = 48

train_generator = datagen.flow_from_directory(
        train_dir_path,  # this is the target directory
        target_size=MNet_InputSize,  # all images will be resized
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

Found 60000 images belonging to 5 classes.


In [ ]:

test_generator = datagen.flow_from_directory(
        test_dir_path,  # this is the target directory
        target_size=MNet_InputSize,  # all images will be resized
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

Found 15000 images belonging to 5 classes.


In [ ]:
history = model.fit_generator(train_generator,
                          epochs=1,
                          validation_data = test_generator,
                          verbose=1)

model.save('rice_image_trained.h5')

<ipython-input-33-b0c24f04d516>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,


 544/1250 [============>.................] - ETA: 29:11 - loss: 0.1380 - accuracy: 0.9499

In [ ]:
if __name__ == "__main__":
    base_dir_path = os.getcwd()
    train_dir_path = os.path.join(base_dir_path,'train')
    test_dir_path = os.path.join(base_dir_path,'test')
    MNet_InputSize = (224,224)

    understandData(base_dir_path)

In [ ]:
#    ImagePath = 'test/Banana Red/99_100.jpg'
#    path_trained_model = "E:/Py_Proj/ML/EXPLORES/deep_object_detect/fruits-360/trained_model/fruit_360_trained_16JUL.h5"
#    trainedModel = getTrainedModel(path_trained_model)
#    AllProbs = predictFruitClass(ImagePath,trainedModel,DictOfClasses)